Pada final project ini teman-teman diminta untuk memprediksi gaji seseorang apakah 7jt berdasarkan beberapa keterangan pada kolom-kolom dataset. Buatlah model klasifikasi yang dapat menentukan nilai pada kolom 'Gaji' dengan ketentuan sebagai berikut:

0 : Gaji <= 7jt
1 : Gaji > 7jt

muhammad Rafi
muhamadrafi23923@gmail.com

In [1]:
import numpy as np
import pandas as pd
import keras
import matplotlib.pyplot as plt
import tensorflow as tf
import csv
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dense, Dropout, Activation, Flatten

Using TensorFlow backend.


In [2]:
df = pd.read_csv('train.csv')
df.head(5)

,id,Umur,Kelas Pekerja,Berat Akhir,Pendidikan,Jmlh Tahun Pendidikan,Status Perkawinan,Pekerjaan,Jenis Kelamin,Keuntungan Kapital,Kerugian Capital,Jam per Minggu,Gaji
0,0,21,Wiraswasta,242912,SMA,9,Belum Pernah Menikah,Servis Lainnya,Perempuan,0.0,0.0,35.0,<=7jt
1,1,49,Wiraswasta,140782,10th,6,Cerai,Ekesekutif Managerial,Perempuan,0.0,0.0,40.0,<=7jt
2,2,44,Wiraswasta,120057,D3,12,Menikah,Ekesekutif Managerial,Laki2,61404000.0,0.0,45.0,>7jt
3,3,24,Wiraswasta,194630,Sarjana,13,Belum Pernah Menikah,Spesialis,Laki2,0.0,0.0,35.0,<=7jt
4,4,33,Wiraswasta,219619,Master,14,Menikah,Spesialis,Laki2,210336000.0,0.0,40.0,>7jt


In [3]:
df.shape

(35994, 13)

In [4]:
df['Gaji'].value_counts()

<=7jt    27364
>7jt      8630
Name: Gaji, dtype: int64

In [5]:
df['Pekerjaan'] = df['Pekerjaan'].replace(
    ['Spesialis','Ekesekutif Managerial','Perbaikan Kerajinan','Pemuka Agama','Sales','Servis Lainnya','Mesin Inspeksi','?','Supir','Pembersih','Petani','Tech-support','Penjaga','Asisten Rumah Tangga','Tentara'],
    [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]
)

In [6]:
df['Pekerjaan'].value_counts()

1     4587
3     4507
2     4481
4     4158
5     4044
6     3615
7     2250
8     2069
9     1741
10    1540
11    1076
12    1042
13     686
14     187
15      11
Name: Pekerjaan, dtype: int64

In [7]:
df['Pendidikan'] = df['Pendidikan'].replace(
    ['SMA','Pendidikan Tinggi', 'Sarjana','Master','D4','11th','D3','10th','7th-8th','Sekolah Professional','9th','12th','Doktor','5th-6th','1st-4th','SD'],
    [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]
)

In [8]:
df['Pendidikan'].value_counts()

1     11666
2      8005
3      5928
4      1944
5      1500
6      1315
7      1175
8      1024
9       682
10      622
11      570
12      478
13      451
14      388
15      184
16       62
Name: Pendidikan, dtype: int64

In [9]:
df['Kelas Pekerja'] = df['Kelas Pekerja'].replace(['Wiraswasta','Pekerja Bebas Bukan Perusahan','Pemerintah Lokal','?','Pemerintah Negara','Pekerja Bebas Perusahaan','Pemerintah Provinsi',
                             'Tanpa di Bayar','Tidak Pernah Bekerja'],[1,2,3,4,5,6,7,8,9])

In [10]:
df['Status Perkawinan'].value_counts()

Menikah                 16559
Belum Pernah Menikah    11851
Cerai                    4857
Berpisah                 1133
Janda                    1103
Menikah LDR               491
Name: Status Perkawinan, dtype: int64

In [11]:
df['Status Perkawinan'] = df['Status Perkawinan'].replace(
    ['Menikah','Belum Pernah Menikah','Cerai','Berpisah','Janda','Menikah LDR'],
    [1,2,3,4,5,6])

In [12]:
df['Jenis Kelamin']=df['Jenis Kelamin'].replace(['Laki2','Perempuan'],[1,0])

In [13]:
df.head(5)

,id,Umur,Kelas Pekerja,Berat Akhir,Pendidikan,Jmlh Tahun Pendidikan,Status Perkawinan,Pekerjaan,Jenis Kelamin,Keuntungan Kapital,Kerugian Capital,Jam per Minggu,Gaji
0,0,21,1,242912,1,9,2,6,0,0.0,0.0,35.0,<=7jt
1,1,49,1,140782,8,6,3,2,0,0.0,0.0,40.0,<=7jt
2,2,44,1,120057,7,12,1,2,1,61404000.0,0.0,45.0,>7jt
3,3,24,1,194630,3,13,2,1,1,0.0,0.0,35.0,<=7jt
4,4,33,1,219619,4,14,1,1,1,210336000.0,0.0,40.0,>7jt


In [14]:
from sklearn.model_selection import train_test_split
x = df[['Umur','Kelas Pekerja','Pendidikan','Pekerjaan','Jmlh Tahun Pendidikan','Status Perkawinan','Jenis Kelamin',
        'Keuntungan Kapital', 'Kerugian Capital','Jam per Minggu']]
y = df['Gaji']
x =np.array(x)
X_train, X_test, y_train, y_test = train_test_split(x,y, test_size = 0.3, random_state=21, stratify = y)

In [ ]:
y_train = keras.utils.to_categorical(y_train, 2)
y_test = keras.utils.to_categorical(y_test,2)

In [18]:
X_test

array([[29.,  1.,  1., ...,  0.,  0., 30.],
       [32.,  1.,  1., ...,  0.,  0., 40.],
       [52.,  1.,  2., ...,  0.,  0., 40.],
       ...,
       [36.,  2.,  1., ...,  0.,  0., 35.],
       [30.,  1.,  1., ...,  0.,  0., 40.],
       [17.,  4.,  8., ...,  0.,  0., 30.]])

In [19]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dense, Dropout, Activation, Flatten
import matplotlib.pyplot as plt
import tensorflow as tf

In [23]:
model = Sequential()
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(2, activation=Activation(tf.nn.softmax)))

In [24]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(), metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, 
          batch_size=28, 
          epochs=50, 
          verbose=1,
          validation_data=(X_test,y_test))

In [59]:
dftrain = pd.read_csv('test.csv')
dftrain.head()

,id,Umur,Kelas Pekerja,Berat Akhir,Pendidikan,Jmlh Tahun Pendidikan,Status Perkawinan,Pekerjaan,Jenis Kelamin,Keuntungan Kapital,Kerugian Capital,Jam per Minggu
0,35994,30,Pemerintah Lokal,99761,D4,11,Cerai,Pemuka Agama,Perempuan,0.0,0.0,40.0
1,35995,28,Wiraswasta,202206,SMA,9,Menikah,Mesin Inspeksi,Laki2,0.0,0.0,50.0
2,35996,34,Pemerintah Provinsi,199934,Pendidikan Tinggi,10,Menikah,Pemuka Agama,Laki2,0.0,0.0,40.0
3,35997,51,Wiraswasta,373448,Pendidikan Tinggi,10,Menikah,Ekesekutif Managerial,Laki2,0.0,28028000.0,40.0
4,35998,20,?,144685,Pendidikan Tinggi,10,Belum Pernah Menikah,?,Perempuan,0.0,22428000.0,40.0


In [60]:
dftrain['Pendidikan'].value_counts()

SMA                     3090
Pendidikan Tinggi       2176
Sarjana                 1564
Master                   509
D4                       417
11th                     379
D3                       312
10th                     271
7th-8th                  214
Sekolah Professional     152
9th                      138
12th                     125
Doktor                   104
5th-6th                   86
1st-4th                   47
SD                        15
Name: Pendidikan, dtype: int64

In [61]:
dftrain['Pendidikan'] = dftrain['Pendidikan'].replace(
    ['SMA','Pendidikan Tinggi', 'Sarjana','Master','D4','11th','D3','10th','7th-8th','Sekolah Professional','9th','12th','Doktor','5th-6th','1st-4th','SD'],
    [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]
)

In [62]:
dftrain['Kelas Pekerja'] = dftrain['Kelas Pekerja'].replace(['Wiraswasta','Pekerja Bebas Bukan Perusahan','Pemerintah Lokal','?','Pemerintah Negara','Pekerja Bebas Perusahaan','Pemerintah Provinsi',
                             'Tanpa di Bayar','Tidak Pernah Bekerja'],[1,2,3,4,5,6,7,8,9])

In [63]:
dftrain['Pekerjaan'] = dftrain['Pekerjaan'].replace(
    ['Spesialis','Ekesekutif Managerial','Perbaikan Kerajinan','Pemuka Agama','Sales','Servis Lainnya','Mesin Inspeksi','?','Supir','Pembersih','Petani','Tech-support','Penjaga','Asisten Rumah Tangga','Tentara'],
    [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]
)

In [64]:
dftrain['Status Perkawinan'] = dftrain['Status Perkawinan'].replace(
    ['Menikah','Belum Pernah Menikah','Cerai','Berpisah','Janda','Menikah LDR'],
    [1,2,3,4,5,6])

In [65]:
dftrain['Jenis Kelamin']=dftrain['Jenis Kelamin'].replace(['Laki2','Perempuan'],[1,0])

In [66]:
predictor = ['Umur','Kelas Pekerja','Pendidikan','Pekerjaan','Jmlh Tahun Pendidikan',
                'Status Perkawinan','Jenis Kelamin','Keuntungan Kapital', 'Kerugian Capital','Jam per Minggu']
hasilprediksi = model.predict(dftrain[predictor])
hasilprediksi

array([[0.7961789 , 0.20382115],
       [0.82485896, 0.17514105],
       [0.82366127, 0.17633879],
       ...,
       [0.7433022 , 0.25669768],
       [0.85668325, 0.14331673],
       [0.8841898 , 0.11581027]], dtype=float32)

In [67]:
def uji(xd):
  a_max = xd.max()
  mylist = xd
  for i in range(len(mylist)):
    if mylist[i] == a_max:
      break
  return i

In [105]:
ok = hasilprediksi[(98)]
ok
uji(ok)

1

In [106]:
ha=[]
for x in range(len(hasilprediksi)):
  ha.append(uji(hasilprediksi[x]))

In [112]:
submit =pd.DataFrame({
    'Id':dftest['id'],
    'Gaji':ha
})
submit.to_csv('submit.csv', index=False)

In [3]:
df = pd.read_csv('submit.csv')
df.head()

,Id,Gaji
0,35994,0
1,35995,0
2,35996,0
3,35997,0
4,35998,0
